# Introduction

Data analysts need to solve business questions daily by directly querying the database. This project will demonstrate the way of answering business questions using SQL.

# Database connection

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [5]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


# Background

The Chinook is a music company in USA. The marketing team would like to get info about the industry, so that the team would make their marketing plan efficiently:
- Top 3 best-selling genre
- Top 3 best-selling artist name

In [82]:
%%sql
WITH invoice_line_track AS 
    (
     SELECT SUM(il.quantity) AS total_quantity_sold,
            t.genre_id
     FROM invoice_line il
     INNER JOIN track t
     ON il.invoice_line_id = t.track_id
     GROUP BY t.genre_id
    )
    
SELECT 
    g.name AS genre, 
    total_quantity_sold FROM invoice_line_track
INNER JOIN genre g
ON invoice_line_track.genre_id = g.genre_id
ORDER BY total_quantity_sold DESC
LIMIT 3;

Done.


genre,total_quantity_sold
Rock,1297
Latin,579
Metal,374


In [83]:
%%sql
WITH invoice_line_track AS 
    (
     SELECT SUM(il.quantity) AS total_quantity_sold,
            t.album_id
     FROM invoice_line il
     INNER JOIN track t
     ON il.invoice_line_id = t.track_id
     GROUP BY t.album_id
    ),
     artist_album AS 
    (
     SELECT artist.name AS artist_name, album_id
     FROM artist
     INNER JOIN album
     ON artist.artist_id = album.album_id
    )
    
SELECT 
    artist_name, 
    total_quantity_sold FROM invoice_line_track ilt
INNER JOIN artist_album aa
ON ilt.album_id = aa.album_id
ORDER BY total_quantity_sold DESC
LIMIT 3;

Done.


artist_name,total_quantity_sold
The Police,57
Frank Zappa & Captain Beefheart,34
Vinícius E Qurteto Em Cy,30


Above results shows that Rock is most popular among the genre, followed by Latin and Metal. In addition, the top 3 best-selling artist names are The Police, Frank Zappa & Captain Beefheart & Vinícius E Qurteto Em Cy.

Furthermore, Sales Manager would like to know:
- Sales trend by year 
- Performance of the team by top 3 sales and their distribution.

In [84]:
%%sql
SELECT strftime('%Y', invoice_date) AS year,
       SUM(total) AS total_sales_amount
FROM invoice
GROUP BY year;

Done.


year,total_sales_amount
2017,1201.8600000000013
2018,1147.4100000000008
2019,1221.6600000000014
2020,1138.5000000000005


In [85]:
%%sql
WITH employee_customer AS 
    (
     SELECT e.employee_id,
     e.first_name || " " || e.last_name AS employee_name,
     c.customer_id
     FROM employee e
     INNER JOIN customer c
     ON e.employee_id = c.support_rep_id
    )
    
SELECT 
    employee_name, 
    SUM(i.total) AS total_sales_amount,
    SUM(i.total) / (SELECT SUM(total) FROM invoice) sales_percentage
FROM employee_customer ec
INNER JOIN invoice i
ON ec.customer_id = i.customer_id
GROUP BY employee_name
ORDER BY total_sales_amount DESC
LIMIT 3;

Done.


employee_name,total_sales_amount,sales_percentage
Jane Peacock,1731.5100000000039,0.3676686987597238
Margaret Park,1584.0000000000034,0.336346436829936
Steve Johnson,1393.920000000002,0.29598486441034344
